In [1]:
from ROOT import TLorentzVector
from ROOT import Math
import numpy as np

Welcome to JupyROOT 6.22/06


In [2]:
def thetaPhi():
    '''returns random theta and phi values'''
    theta = np.random.uniform(0,2*np.pi)
    phi = np.random.uniform(0,np.pi)
    return theta, phi

def momenta(mag, theta, phi):
    '''given an input magnitude, theta and phi the cartesian momenta are returned'''
    Px = mag * np.sin(theta) * np.cos(phi) 
    Py = mag * np.sin(theta) * np.sin(phi)
    Pz = mag * np.cos(theta)
    
    return Px, Py, Pz
    

def wArr(num):
    '''just for making numpy array of stationary w given the number of events'''
    momenta = np.zeros([num,4])
    momenta[:,3] = wMass
    
    return momenta

def vecToArr(vec):
    '''converts lorentzvector to array of components'''
    arr = np.zeros([4])
    arr[0] = vec.Px()
    arr[1] = vec.Py()
    arr[2] = vec.Pz()
    arr[3] = vec.E()
    
    return arr

In [3]:
def backToBack(beta):
    '''from a given beta (ratio v/c) of the initial particles produces numpy array of 4 momenta of
    w pair with one hadronic decay and one leptonic decay'''
    event = np.zeros([6,4])
    # making back to back w bosons (stationary)
    wMass = 80.3
    
    wmag = 3 # ??
    
    theta, phi = thetaPhi()
    Px, Py, Pz = momenta(wmag, theta, phi)

    w1 = Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(Px, Py, Pz, wMass)
    w2 = Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(-Px, -Py, -Pz, wMass)

    #boosting the Ws
    beta = beta
    boostedW1 = Math.VectorUtil.boostZ(w1, beta)
    w1Arr = vecToArr(boostedW1)
    boostedW2 = Math.VectorUtil.boostZ(w2, beta)
    w2Arr = vecToArr(boostedW2)

    # have to make them into tlorentzvectors for the boostvector function to work
    boow1 = TLorentzVector()
    boow1.SetPxPyPzE(boostedW1.Px(), boostedW1.Py(), boostedW1.Pz(), boostedW1.E())
    boow2 = TLorentzVector()
    boow2.SetPxPyPzE(boostedW2.Px(), boostedW2.Py(), boostedW2.Pz(), boostedW2.E())

    #making boost vectors for the daughter particles of each w
    boost1 = boow1.BoostVector()
    boost2 = boow2.BoostVector()


    #making the hadronic decay particles this will use w1 as its parent
    #in w1 rest frame to start
    ej = wMass/2
    thetaj, phij = thetaPhi()
    Px, Py, Pz = momenta(ej, thetaj, phij)
    j1 = Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(Px, Py, Pz, ej)
    j2 = Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(-Px, -Py, -Pz, ej)

    # boosting jets to frame of W1
    j1Boost = Math.VectorUtil.boost(j1, boost1)
    j1Arr = vecToArr(j1Boost)
    j2Boost = Math.VectorUtil.boost(j2, boost1)
    j2Arr = vecToArr(j2Boost)

    # making leptonic decay particles using W2 as parent
    # w2 rest frame to start
    eE = wMass / 2
    thetae, phie = thetaPhi()
    Px, Py, Pz = momenta(eE, thetae, phie)
    e = Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(Px, Py, Pz, eE)
    n = Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(-Px, -Py, -Pz, eE)

    #boosting e and n to frame of W2
    eBoost = Math.VectorUtil.boost(e, boost2)
    eArr = vecToArr(eBoost)
    nBoost = Math.VectorUtil.boost(n, boost2)
    nArr = vecToArr(nBoost)

    #putting all particles into one array
    event[0] = w1Arr
    event[1] = w2Arr
    event[2] = j1Arr
    event[3] = j2Arr
    event[4] = eArr
    event[5] = nArr
    
    return event

for the array 'event' which comes from above function the first entry is the w1 4 momenta, the 2nd is the w2 momenta, 3rd is jet 1, 4th is jet 2, 5th is the electron, and 6th is the neutrino

In [4]:
print(backToBack(0.3))

[[  0.8138998   -2.34533545  27.01883969  84.70696978]
 [ -0.8138998    2.34533545  23.48752374  83.647575  ]
 [ 12.03829073  -9.08668193 -26.04271683  30.09503833]
 [-11.22382234   6.739708    53.08043222  54.6711088 ]
 [  9.08337975 -16.89928902 -24.45923372  31.08613632]
 [ -9.89784814  19.24626294  47.96316614  52.61987592]]


In [5]:
# looping this function can give big array of data
num = 10
events = np.zeros([num, 6, 4])
beta = 0.1

for i in range(num):
    events[i] = backToBack(beta)

In [6]:
print(events)

[[[ 2.51050773e+00  1.64207791e+00  8.03978634e+00  8.07014698e+01]
  [-2.51050773e+00 -1.64207791e+00  8.10112097e+00  8.07076033e+01]
  [-9.42766108e-01 -1.02109403e+00 -3.62356904e+01  3.62623317e+01]
  [ 3.45502771e+00  2.66431912e+00  4.42810934e+01  4.44955172e+01]
  [ 2.97035314e+01  2.38403685e+01 -3.02039795e+00  3.82071427e+01]
  [-3.22157930e+01 -2.54835936e+01  1.11271785e+01  4.25568439e+01]]

 [[-2.56517111e+00  2.69079919e-02  9.63365845e+00  8.08608570e+01]
  [ 2.56517111e+00 -2.69079919e-02  6.50724886e+00  8.05482161e+01]
  [-1.86468464e+01 -4.71494592e+00 -3.12663842e+01  3.67086145e+01]
  [ 1.60798833e+01  4.74187271e+00  4.09067729e+01  4.42087329e+01]
  [ 3.07454597e+01 -1.29724124e+01 -2.07665132e+01  3.93041327e+01]
  [-2.81784965e+01  1.29454856e+01  2.72783081e+01  4.13003554e+01]]

 [[-1.16461461e+00  6.60604169e-01  1.07686152e+01  8.09743527e+01]
  [ 1.16461461e+00 -6.60604169e-01  5.37229211e+00  8.04347204e+01]
  [ 7.40050670e+00  8.79483307e+00  4.415343